In [2]:
import cv2 as cv
from deepface import DeepFace
import time
import numpy as np


2023-10-30 15:49:32.565506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 15:49:32.644449: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
# Define backends and models
backends = [
  'opencv',
  "retinaface" 
]

# Direct sentiment analysis

In [4]:
def draw(image, bounding_boxes, text):
    i = 1
    for bbox in bounding_boxes:
        x = bbox['region']['x']
        y = bbox['region']['y']
        w = bbox['region']['w']
        h = bbox['region']['h']

        # Draw a rectangle on the image
        img = cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # (0, 255, 0) is the color in BGR format
        img = cv.putText(img, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        i+=1
    return img 

In [7]:
video_file_path = 'experimentation_videos/videoplayback.mp4'  # Replace with the path to your video file
for i, backend in enumerate(backends):
    cap = cv.VideoCapture(video_file_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(f'model_output/experiment_3_{i}.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
    if not cap.isOpened():
        print("Error: Could not open video file.")

    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break
            
        objs = DeepFace.analyze(frame,
                actions = ['emotion'],
                enforce_detection=False,
                silent=True,
                detector_backend=backend
        )

        frame = draw(frame, objs, objs[0]["dominant_emotion"])
        out.write(frame)
        # cv.imshow("Yolo", frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release() # Release the video file
    cv.destroyAllWindows()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"For {backend}, Elapsed time: {elapsed_time} seconds")


For opencv, Elapsed time: 1365.808161020279 seconds
For retinaface, Elapsed time: 2602.1890172958374 seconds


: 